# Clasificador de Fuerza de Contraseña

Este cuaderno demuestra el proceso de construcción de un clasificador de fuerza de contraseña utilizando aprendizaje automático. El objetivo es entrenar un modelo que pueda predecir si una contraseña dada es débil, media o fuerte basándose en sus características.

El proceso involucra los siguientes pasos:

1.  **Carga y Preparación de Datos:** Cargar el conjunto de datos que contiene contraseñas y sus etiquetas de fuerza correspondientes. Los datos se leen desde un archivo CSV y se realiza una limpieza básica.
2.  **Extracción de Características:** Convertir las contraseñas basadas en texto en características numéricas que puedan ser utilizadas por un modelo de aprendizaje automático. Esto se realiza utilizando la técnica de vectorización TF-IDF (Frecuencia de Término - Frecuencia Inversa de Documento).
3.  **Entrenamiento del Modelo:** Entrenar un modelo de clasificación (Random Forest Classifier) en los datos preparados para aprender la relación entre las características de la contraseña y su fuerza.
4.  **Evaluación del Modelo:** Evaluar el rendimiento del modelo entrenado en un conjunto de prueba separado para evaluar su precisión en la predicción de la fuerza de la contraseña.
5.  **Predicción:** Utilizar el modelo entrenado para predecir la fuerza de contraseñas nuevas y no vistas.

El cuaderno utiliza varias librerías de Python como `pandas` para manipulación de datos, `numpy` para operaciones numéricas, `sklearn` para tareas de aprendizaje automático (incluyendo extracción de características, selección y entrenamiento de modelos) y `tqdm` para mostrar barras de progreso.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Importar dependencias

# La librería JSON permite abrir y guardar archivos en formato JSON
import json
# Sklearn cuenta con una herramienta para dividir un dataset
# en un subconjunto de entrenamiento y un subconjunto de evaluación
# Ambos, son importantes para el trabajo con aprendizaje automatico
from sklearn.model_selection import train_test_split
# os es una librería propia de Python que permite precesar archivos
import os
# La librería TQDM permite integrar barras de progreso en estructuras repetitivas for
from tqdm.auto import tqdm
# La librería shutil tiene implementaciones optimizadas para trabajar con archivos
import shutil
# La librería random permite crear números aleatorios
import random
import cv2
from matplotlib import pyplot as plt
from google.colab import userdata
import seaborn as sns

En esta seccion se configura las credenciales de Kaggle en tu entorno de Colab para permitir la descarga de datasets directamente desde Kaggle. Los pasos son:

*   **Definir la ruta:** Establece la ubicación donde se guardarán las credenciales (`/root/.kaggle`).
*   **Limpiar y crear directorio:** Elimina el directorio de Kaggle si ya existe y luego lo crea de nuevo para asegurar una configuración limpia.
*   **Crear archivo de credenciales:** Crea un archivo vacío `kaggle.json` dentro del directorio.
*   **Generar token de API:** Crea un diccionario con tu nombre de usuario de Kaggle (reemplaza "USERNAME") y tu clave de API obtenida de los secretos de Colab (`userdata.get('kaggle')`).
*   **Guardar credenciales:** Escribe el diccionario con las credenciales en el archivo `kaggle.json` en formato JSON.
*   **Establecer permisos:** Cambia los permisos del archivo `kaggle.json` a `600` para proteger tus credenciales, permitiendo solo al propietario leer y escribir.

In [2]:
# Definir las credentiales para acceder a Kaggle. Primero requerimos confirmar que
# el archivo aún no ha sido creado
KAGGLE_PATH = "/root/.kaggle"
if os.path.exists(KAGGLE_PATH):
  !rm -r "$KAGGLE_PATH"

!mkdir "$KAGGLE_PATH"
!touch "$KAGGLE_PATH/kaggle.json"

# Puedes crear tu propio token y username de la API de Kaggle en https://www.kaggle.com/
api_token = {"username":"USERNAME","key":userdata.get('kaggle')}

# Crear un archivo con las credenciales, de tal forma que kaggle pueda leerlas facilmente
with open(KAGGLE_PATH+'/kaggle.json', 'w') as file:
    json.dump(api_token, file)

# Cambiar los permisos de acceso del nuevo archivo con credenciales
!chmod 600 ~/.kaggle/kaggle.json

En esta seccion se descarga el conjunto de datos de contraseñas desde Kaggle a tu entorno en Colab.

*   Primero, se asegura de estar en la carpeta principal (`/content`).
*   Luego, borra la carpeta donde se guardará el dataset si ya existía, y crea una nueva.
*   Finalmente, descarga el archivo comprimido del dataset de contraseñas a esa carpeta.

In [3]:
%cd /content

# Comprobar si el conjunto de datos ya se ha descargado
if os.path.exists('./Password_Classifier'):
  !rm -rf Password_Classifier
os.makedirs('Password_Classifier')
# Descargar un dataset desde Kaggle
!kaggle datasets download -d bhavikbb/password-strength-classifier-dataset -p Password_Classifier

/content
Dataset URL: https://www.kaggle.com/datasets/bhavikbb/password-strength-classifier-dataset
License(s): copyright-authors
  0% 0.00/5.01M [00:00<?, ?B/s]
100% 5.01M/5.01M [00:00<00:00, 1.65GB/s]


En esta sección se descomprime el archivo del conjunto de datos de contraseñas que se descargó previamente.

*   Se extrae el contenido del archivo `.zip` a la carpeta `Password_Classifier`.
*   Luego, se elimina el archivo `.zip` original para ahorrar espacio.

In [4]:
# Descomprimir dataset
!unzip -qn '/content/Password_Classifier/password-strength-classifier-dataset.zip' -d /content/Password_Classifier
!rm /content/Password_Classifier/password-strength-classifier-dataset.zip

En esta sección se carga el conjunto de datos desde el archivo CSV (`data.csv`) a un DataFrame de pandas llamado `data`.

*   `pd.read_csv(...)`: Esta función de pandas se utiliza para leer datos desde un archivo CSV.
*   `"/content/Password_Classifier/data.csv"`: Es la ruta al archivo CSV que descargamos y descomprimimos anteriormente.
*   `delimiter=','`, `quoting=3`, `engine='python'`, `on_bad_lines='skip'`: Estos argumentos ayudan a pandas a leer el archivo correctamente, especificando que las columnas están separadas por comas, cómo manejar las comillas, usar el motor de análisis de Python para mayor robustez y saltar las líneas que no se puedan leer correctamente.

In [5]:
data = pd.read_csv("/content/Password_Classifier/data.csv", delimiter=',', quoting=3, engine='python', on_bad_lines='skip')

In [6]:
# Ver las columnas y las filas
data.head(20)

,password,strength
0,kzde5577,1
1,kino3434,1
2,visi7k1yr,1
3,megzy123,1
4,lamborghin1,1
5,AVYq1lDE4MgAZfNt,2
6,u6c8vhow,1
7,v1118714,1
8,universe2908,1
9,as326159,1


En esta sección se limpian y transforman los datos para prepararlos para el modelo.

*   Primero, se eliminan las filas que contienen valores faltantes para asegurar que no haya datos incompletos.
*   Luego, se cambian las etiquetas numéricas de la columna 'strength' (0, 1, 2) por etiquetas de texto más descriptivas ("Weak", "Medium", "Strong") para facilitar la comprensión de los resultados.
*   Finalmente, se muestra una pequeña muestra aleatoria de los datos transformados para ver cómo quedaron.

In [7]:
data = data.dropna()
data["strength"] = data["strength"].map({0: "Weak",
                                         1: "Medium",
                                         2: "Strong"})
print(data.sample(5))

            password strength
560664       ekouhv4     Weak
150551  eder01061977   Medium
444280    iwyqipy818   Medium
197240      sobiesk1   Medium
25995      ghsdukes1   Medium


En esta sección se preparan los datos para el modelo de aprendizaje automático.

*   Se define una función simple (`word`) para dividir cada contraseña en caracteres individuales.
*   Se separan las contraseñas (`x`) de sus etiquetas de fuerza (`y`).
*   Se utiliza `TfidfVectorizer` con la función `word` para convertir las contraseñas (texto) en vectores numéricos (características) que el modelo pueda entender. Esto se llama vectorización TF-IDF.
*   Finalmente, se divide el conjunto de datos vectorizado en conjuntos de entrenamiento (`xtrain`, `ytrain`) y prueba (`xtest`, `ytest`). El conjunto de entrenamiento se usa para enseñar al modelo, y el conjunto de prueba para evaluar qué tan bien aprendió. Se usa un 5% de los datos para prueba.

In [8]:
def word(password):
    character=[]
    for i in password:
        character.append(i)
    return character

x = np.array(data["password"])
y = np.array(data["strength"])

tdif = TfidfVectorizer(tokenizer=word)
x = tdif.fit_transform(x)
xtrain, xtest, ytrain, ytest = train_test_split(x, y,
                                                test_size=0.05,
                                                random_state=42)

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


En esta sección se entrena el modelo de clasificación y se evalúa su rendimiento.

*   **`model = RandomForestClassifier()`**: Se crea una instancia del modelo `RandomForestClassifier`. Este modelo es un algoritmo de aprendizaje automático basado en árboles de decisión que es adecuado para tareas de clasificación.
*   **`model.fit(xtrain, ytrain)`**: Se entrena el modelo utilizando los datos de entrenamiento (`xtrain` y `ytrain`). El modelo aprende a asociar las características de las contraseñas con sus etiquetas de fuerza.
*   **`print(model.score(xtest, ytest))`**: Se evalúa el rendimiento del modelo entrenado utilizando los datos de prueba (`xtest` y `ytest`). El método `score` para clasificadores devuelve la precisión media del modelo en los datos de prueba, es decir, el porcentaje de contraseñas cuya fuerza predijo correctamente.

In [9]:
model = RandomForestClassifier()
model.fit(xtrain, ytrain)
print(model.score(xtest, ytest))

0.9564542142046473


En esta sección, puedes probar el modelo entrenado con una contraseña que ingreses.

*   **`import getpass`**: Importa la librería `getpass` para poder solicitar la contraseña de forma segura sin que se muestre en la pantalla mientras escribes.
*   **`user = getpass.getpass("Enter Password: ")`**: Le pide al usuario que ingrese una contraseña. El texto "Enter Password: " se muestra como un mensaje.
*   **`data = tdif.transform([user]).toarray()`**: Aquí se prepara la contraseña ingresada por el usuario para que el modelo la entienda. Primero, se transforma la contraseña de texto a un formato numérico utilizando el mismo vectorizador TF-IDF (`tdif`) que se usó para entrenar el modelo. `[user]` se usa porque `transform` espera una lista de contraseñas, incluso si es solo una. `.toarray()` convierte el resultado a un formato que el modelo pueda procesar.
*   **`output = model.predict(data)`**: Se utiliza el modelo entrenado (`model`) para predecir la fuerza de la contraseña transformada (`data`). El resultado de la predicción se guarda en la variable `output`.
*   **`print(output)`**: Imprime el resultado de la predicción, que será la fuerza de la contraseña clasificada como "Weak", "Medium" o "Strong".

In [11]:
import getpass
user = getpass.getpass("Enter Password: ")
data = tdif.transform([user]).toarray()
output = model.predict(data)
print(output)

Enter Password: ··········
['Strong']
